In [12]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'))
documents = dataset.data

In [4]:
no_features = 1000

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=no_features,
                                   stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [5]:
from minisom import MiniSom

In [37]:
som = MiniSom(2, 4, no_features)

In [38]:
D = tfidf.todense().tolist()
som.pca_weights_init(D)
som.train_batch(D, 40000)

In [70]:
top_keywords = 10

weights = som.get_weights()
cnt = 1
for i in range(2):
    for j in range(4):
        keywords_idx = np.argsort(weights[i,j,:])[-top_keywords:]
        keywords = ' '.join([tfidf_feature_names[k] for k in keywords_idx])
        print('Topic', cnt, ':', keywords)
        #plt.text(i, j, tfidf_feature_names[])
        #plt.plot(i, j, '.')
        cnt += 1

Topic 1 : don like hear good think year game did games team
Topic 2 : day know like people religion said law don didn evidence
Topic 3 : use like word speed digital mouse need sure know ll
Topic 4 : file set use files driver printer problem pc windows mode
Topic 5 : good assume people want clinton read just book money like
Topic 6 : good want know right like god does don jesus people
Topic 7 : don use air cards chips stuff com sun does know
Topic 8 : info use new appreciated help scsi drive mail advance thanks
